# Data Wrangling

## Structure

The data wrangling process goes through 8 stages:

__1.__ Load data from source jsonl files <br><br> 
__2.__ Filter data for extractive summaries only<br><br> 
__3.1__ Preprocess data with spacy to form cleaned sentence lists for each article <br><br>
__3.2__ Use sentence tranformer to calcuate BERT sentence embeddings as the features set <br><br>
__4.__ Calculate target variables by finding highest cosine similarity article sentence for each summary sentence <br><br> 
__5.__ Add additional features (sentence number and document length) <br><br>
__6.__ Add subject domain labels for each article <br><br>
__7.__ Preprocess for train_test_split on embeddings features set only with split at document level <br><br>
__8.__ Add sentence number to embeddings features for another train_test_split set <br><br>


## 1. Load Data

Load data from source jsonl files

In [ ]:
"""
load_data_from_jsonl.py
"""
import json
import pandas as pd
import pickle

input_file = 'train-stats.jsonl'
output_file = 'train_stats_df_no_spacy.pickle'

#read jsonl file into list of sample rows
counter=0
data=[]
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))
        counter +=1
        
#wrap in dataframe        
df = pd.DataFrame(data)

#save to pickle
with open(output_file, 'wb') as handle:                                     
    pickle.dump(df, handle)


## 2. Filter Data

Filter data for extractive summaries only

In [ ]:
"""
filter_extractive_data_only.py
""" 
import pickle
import pandas as pd
 
output_file = 'train_stats_df_extractive_no_spacy.pickle'

#load all data
df = pd.read_pickle('train_stats_df_no_spacy.pickle')

#filter for extractive summaries only
df = df[df.density_bin == 'extractive']

#save to pickle file
with open(output_file, 'wb') as handle:                                     
    pickle.dump(df, handle)


## 3. Preprocessing: Sentence Tokenization and Embeddings (Features)

3.1 Preprocess data with spacy to form cleaned sentence lists for each article <br> 
3.2 Use sentence tranformer to calcuate BERT sentence embeddings as the features set

In [ ]:
"""
preprocessing_embed.py
"""
import pickle
import pandas as pd
import numpy as np
import spacy
from sentence_transformers import SentenceTransformer
from datetime import datetime as dt


### Helper function

def text_to_sent_list(text, 
                      nlp = spacy.load("en_core_web_lg"), 
                      embedder = SentenceTransformer('distilbert-base-nli-mean-tokens'),
                      min_len=2):
    
    ''' Returns cleaned article sentences and BERT sentence embeddings'''
    
    #convert to list of sentences
    text = nlp(text)
    sents = list(text.sents)
    #remove short sentences by threshhold                                                                                                
    sents_clean = [sentence.text for sentence in sents if len(sentence)> min_len]
    #remove entries with empty list
    sents_clean = [sentence for sentence in sents_clean if len(sentence)!=0]
    #embed sentences (deafult uses BERT SentenceTransformer)
    sents_embedding= np.array(embedder.encode(sents_clean, convert_to_tensor=True))
    
    return sents_clean, sents_embedding



### Script

output_file = 'train_stats_df_processed_extr_5000.pickle'  
#load full extractive df
df = pd.read_pickle('train_stats_df_extractive_no_spacy.pickle')

#truncate for local computation
df= df.head(5000).reset_index(drop=True)

#load nlp and embedder
nlp = spacy.load("en_core_web_lg")
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')

t1 = dt.now()
print(t1)

#extract clean sentence list and sentence embedding for each article TEXT
f = lambda text: text_to_sent_list(text, nlp=nlp, embedder=embedder, min_len=2)
s_interim_tuple = df['text'].apply(f)

df['text_clean'] = s_interim_tuple.apply(lambda x: x[0])
df['text_embedding'] = s_interim_tuple.apply(lambda x: x[1])

#extract clean sentence list and sentence embedding for each article SUMMARY
f = lambda summ: text_to_sent_list(summ, nlp=nlp, embedder=embedder, min_len=0)
s_interim_tuple = df['summary'].apply(f)

df['summary_clean'] = s_interim_tuple.apply(lambda x: x[0])
df['summary_embedding'] = s_interim_tuple.apply(lambda x: x[1])

with open(output_file, 'wb') as handle:                                     
    pickle.dump(df, handle)

t2=dt.now()
print(t2)
print(t2-t1)

#5000 articles took 1hr40mins

## 4. Calculating Target Labels

Calculate target variables by finding highest cosine similarity article sentence for each summary sentence

In [ ]:
"""
preprocessing label_target.py
"""
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

from datetime import datetime as dt

### Helper Functions

def find_sim_single_summary(summary_sentence_embed, doc_emedding):
    '''returns array of indices for max cosine sim per summary sentences'''
    cos_sim_mat = cosine_similarity(doc_emedding, summary_sentence_embed)
    idx_arr = np.argmax(cos_sim_mat, axis=0)
    
    return idx_arr

def label_sent_in_summary(s_text, s_summary):
    '''returns index list and binary target labels in an array'''
    doc_num = s_text.shape[0]
    
    #initialize zeros
    labels = [np.zeros(doc.shape[0]) for doc in s_text.tolist()] 
    
    #calc idx for most similar
    
    idx_list = [np.sort(find_sim_single_summary(s_summary[j], s_text[j])) for j 
                                                            in range(doc_num)]
      
    for j in range(doc_num):
        labels[j][idx_list[j]]= 1 
    
    return idx_list, labels


### Script

t1 = dt.now()
print(t1)

output_file = 'train_stats_df_processed_extr_label_5000.pickle'

df = pd.read_pickle('train_stats_df_processed_extr_5000.pickle' )

#get index list and target labels
idx_list, labels = label_sent_in_summary(df.text_embedding, df.summary_embedding)

#wrap in dataframe
df['labels'] = labels
df['labels_idx_list'] = idx_list

#save to pickle
with open(output_file, 'wb') as handle:                                     
    pickle.dump(df, handle)

t2 = dt.now()

print(t2)

print(t2-t1)


## 5. Additional Features

Add additional features (sentence number and document length)

In [ ]:
"""
add_additional_features.py
"""

import numpy as np
import pandas as pd
import pickle
from datetime import datetime as dt

t1 = dt.now()
print(t1)
    
output_file = 'train_stats_dict_processed_extr_final_5000_.pickle'
    
df = pd.read_pickle('train_stats_df_processed_extr_label_5000.pickle')

#define features and labels as separate series
s_embed_text = df.text_embedding
s_y_labels= df.labels

#label docs
s_doc_label = pd.Series(range(df.shape[0]), name = 'doc_label')

#calculate doc mean
s_doc_mean = s_embed_text.apply(lambda x: x.mean(axis=0).reshape(1,-1))
    
#calculate doc sent length
s_doc_length = s_embed_text.apply(lambda x: x.shape[0])


#create values for each sentence in doc 
X_doc_label_list =[]
X_doc_mean_list = []
X_doc_length_list = []
X_sent_num_list = []

for j in range(len(df)):
    X_doc_label = s_doc_label[j]
    X_doc_mean = s_doc_mean[j]
    X_doc_length = s_doc_length[j]
    X_text = s_embed_text [j]
    n = X_text.shape[0]
    
    X_doc_label_fixed = X_doc_label
    X_doc_mean_fixed = X_doc_mean
    X_doc_length_fixed = X_doc_length 
    sent_num = []
    for i in range(n-1): 
        X_doc_label = np.vstack((X_doc_label, X_doc_label_fixed )) 
        X_doc_mean = np.vstack((X_doc_mean, X_doc_mean_fixed )) 
        X_doc_length = np.vstack((X_doc_length, X_doc_length_fixed )) 
        sent_num.append(i)
    sent_num.append(n-1)
    
    X_doc_label_list.append(X_doc_label)
    X_doc_mean_list.append(X_doc_mean)
    X_doc_length_list.append(X_doc_length)
    X_sent_num_list.append(np.array(sent_num).reshape(-1,1))
    
#from list to pandas series
s_doc_label = pd.Series(X_doc_label_list)
s_doc_mean = pd.Series(X_doc_mean_list)
s_doc_length = pd.Series(X_doc_length_list)
s_sent_num = pd.Series(X_sent_num_list)

#concatenate documents with rows = sentences
  #intialize
Xy_doc_label = s_doc_label.values[0]
X = np.hstack((s_embed_text[0], s_doc_mean[0], s_sent_num[0], s_doc_length[0]))
y= s_y_labels[0].reshape(-1,1)
  #recursive population
f = np.vectorize(lambda x: x if type(x) == np.ndarray else np.array([[x]]))  
for j in range(1, len(df)):
    Xy_doc_label_new = s_doc_label.values[j]
    
    X_text_new = s_embed_text [j]
    X_sent_num_new = s_sent_num[j]
    X_doc_mean_new = s_doc_mean[j]
    X_doc_length_new = f(s_doc_length[j])
    y_new = s_y_labels[j].reshape(-1,1)
    
    X_new = np.hstack((X_text_new, X_doc_mean_new, X_sent_num_new, X_doc_length_new))
    
    X = np.vstack((X, X_new))
    y = np.vstack((y, y_new))           
    
    Xy_doc_label = np.vstack((Xy_doc_label, Xy_doc_label_new))
        
#wrap X in dataframe with lables
labels_text_embedding = ['Sent_BERT_D_' + str(j) for j in range(768)]
labels_doc_mean = ['Doc_BERT_D_' + str(j) for j in range(768)]
other_labels = ['Sent_Number', 'Doc_Length']
col_names = labels_text_embedding + labels_doc_mean + other_labels

df_X = pd.DataFrame(X, columns = col_names)
    
data_dict = {'df_original': df, 'Xy_doc_label_array': Xy_doc_label, 
              'df_X': df_X, 'y_array': y}
    
with open(output_file, 'wb') as handle:                                     
    pickle.dump(data_dict, handle)
    
t2 = dt.now()

print(t2)
print(t2-t1)

#5000 articles took 48 mins

## 6. Subject Domain Labels

Add subject domain labels for each article

In [ ]:
"""
add_domain_labels.py
"""

import pandas as pd
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

output_file = 'extractive_all_domain_labels.pickle' 

input_file = 'train_stats_dict_processed_extr_final_5000_.pickle' 
data = pd.read_pickle(input_file )

embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')

#Make single df with only Embeddings and doc label
df_embed = data['df_X'].loc[:,'Doc_BERT_D_0': 'Doc_BERT_D_767']
df_doc_label = pd.DataFrame(data['Xy_doc_label_array'],columns=['doc_label'])
df = pd.concat([df_doc_label, df_embed], axis=1)
df = df.drop_duplicates().set_index('doc_label', drop=True)

#embed lambda function
embed = lambda x: embedder.encode(x, convert_to_tensor=False)

#define subject domains
domains = ['entertainment','politics', 'business', 'crime']
#find domain word embeddings using BERT
domain_embed = [embed(dom) for dom in domains]
#wrap in dataframe
df_dom_embed = pd.DataFrame(domain_embed, index = domains,
                            columns = df.columns)
#calculate cosine similarity between article and each subject
cos_matrix = cosine_similarity(df, df_dom_embed)

#return subject word from index number function
f = np.vectorize(lambda x: domains[x])
#find max cos sim and return matching subject
doc_domain = f(np.argmax(cos_matrix, axis=1))
#Add to primary dataframe
df['domain'] = doc_domain

#Add to primary dictionary for storage
data.update({'domain_labels_arr': df['domain'].values})

#save to pickle file
with open(output_file, 'wb') as handle:                                     
    pickle.dump(data, handle)

## 7. Train_test_split 1: Embedding Features Only

Preprocess for train_test_split on embeddings features set only with split at document level

In [ ]:
### Helper Function

'''functions.py'''

def gen_train_test_split_doc_level(Xy_doc_label, X, y, 
                                         test_ratio, folds=1, rand_seed=42):
    '''returns train doc labels, test doc labels, and train and test sets
    for features X and target Y'''
    
    
    random.seed(rand_seed)
    
    #index is doc label 
    total_docs = Xy_doc_label.max()
    train_docs_num = int(total_docs*(1-test_ratio))
    
    #for k >1, want to ensure different seeds
    rand_state_list = random.sample(range(2*folds), folds)
    
    #look through k folds
    train_test_set = []
    for state in rand_state_list:
    
        random.seed(state)
        #sample random training set and mask
        train_docs = random.sample(range(1, total_docs+1), train_docs_num)
        train_mask = np.array([x in train_docs for x in list(Xy_doc_label)])
        
        #use mask to define train and test sets
        X_train = X[train_mask]
        y_train = y[train_mask]
    
        X_test = X[~train_mask]
        y_test = y[~train_mask]
    
        Xy_doc_label_train = Xy_doc_label[train_mask]
        Xy_doc_label_test = Xy_doc_label[~train_mask]
        
        #assign all data to tuple for each pass
        data_pass = (Xy_doc_label_train, Xy_doc_label_test,
                                             X_train, X_test, y_train, y_test)
        #append results for ith fold to set 
        train_test_set.append(data_pass)
    
    #set answer tuples to final tuple as container
    train_test_set = tuple(train_test_set)

    return train_test_set

In [ ]:
"""
preprocess_train_test_split_embeddings_only.py
"""
import pickle
import pandas as pd
from functions import gen_train_test_split_doc_level

input_filename = 'extractive_all_domain_labels.pickle' 
folds = 1

output_file = 'train_test_set20_embeddings_only.pickle'

data_dict = pd.read_pickle(input_filename)

#Specify model inputs: df, X, y, doc_labels
df = data_dict['df_original']
Xy_doc_label = data_dict['Xy_doc_label_array']
X = data_dict['df_X'].drop(['Sent_Number','Doc_Length'], axis=1).values
y = data_dict['y_array']

        
#train test split at document level

train_test_set = gen_train_test_split_doc_level(Xy_doc_label, X, y, 
                                         test_ratio=0.2, folds=folds, rand_seed=42)

data_dict.update({'train_test_sets': train_test_set })

with open(output_file, 'wb') as handle:                                     
    pickle.dump(data_dict, handle)

## 8. Train_test_split 2: Embedding and Num Sentence Features

Add sentence number to embeddings features for another train_test_split set

In [ ]:
"""
add_sent_num_to_train_test_split.py
"""
import pickle
import pandas as pd
from functions import gen_train_test_split_doc_level


input_filename = 'extractive_all_domain_labels.pickle' 
folds = 1

output_file = 'train_test_set20_embeddings_sent_num.pickle'

data_dict = pd.read_pickle(input_filename)

#Specify model inputs: df, X, y, doc_labels
df = data_dict['df_original']
Xy_doc_label = data_dict['Xy_doc_label_array']
X = data_dict['df_X'].drop(['Doc_Length'], axis=1).values
y = data_dict['y_array']

        
#train test split at document level

train_test_set = gen_train_test_split_doc_level(Xy_doc_label, X, y, 
                                         test_ratio=0.2, folds=folds, rand_seed=42)

data_dict.update({'train_test_sets': train_test_set })

with open(output_file, 'wb') as handle:                                     
    pickle.dump(data_dict, handle)